In [1]:
import os
import pandas as pd
import json
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
key=os.getenv("OPENAI_API_KEY")

In [4]:
llm=ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.6)

In [5]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback


In [6]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [7]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [8]:
quiz_gen_prompt=PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE
)

In [9]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_gen_prompt,output_key='quiz',verbose=True)

In [10]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [11]:
quiz_evlauation_prompt=PromptTemplate(
    input_variables=['subject','quiz'],
    template=TEMPLATE
)

In [12]:
evlation_chain=LLMChain(llm=llm,prompt=quiz_evlauation_prompt,output_key="review",verbose=True)

In [19]:
mcq_chains=SequentialChain(chains=[quiz_chain,evlation_chain],input_variables=['text','number','subject','tone','response_json'],
                           output_variables=["quiz", "review"], verbose=True,)

In [26]:
file_path=r"E:\mcq_generator_OpenAI\data.txt"
file_path

'E:\\mcq_generator_OpenAI\\data.txt'

In [27]:
with open(file_path,'r') as file:
    text=file.read()

In [28]:
text

"Virat Kohli (Hindi pronunciation; born 5 November 1988) is an Indian international cricketer and the former captain of the Indian national cricket team. He currently represents Royal Challengers Bangalore in the IPL and Delhi in domestic cricket. Kohli is widely regarded as one of the greatest batsmen in the history of cricket, and the best of this era.[3] He is the highest run scorer in T20I and IPL. In 2020, the International Cricket Council named him the male cricketer of the decade. Kohli is currently fourth-highest run-scorer in international cricket and stands second in the list of most international centuries scored. He also holds the record for scoring the most centuries in One Day International cricket.[4][5] Kohli was a member of the Indian team that won the 2011 Cricket World Cup and 2013 ICC Champions Trophy.\n\nIn 2013, Kohli was ranked number one in the ICC rankings for ODI batsmen. In 2018, he was ranked top Test batsman, making him the only Indian cricketer to hold the

In [29]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [33]:
NUMBER= 5
SUBJECT= 'cricket'
TONE= "simple"

In [34]:
with get_openai_callback() as cb:
    response=mcq_chains(
        {
            "text": text,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Virat Kohli (Hindi pronunciation; born 5 November 1988) is an Indian international cricketer and the former captain of the Indian national cricket team. He currently represents Royal Challengers Bangalore in the IPL and Delhi in domestic cricket. Kohli is widely regarded as one of the greatest batsmen in the history of cricket, and the best of this era.[3] He is the highest run scorer in T20I and IPL. In 2020, the International Cricket Council named him the male cricketer of the decade. Kohli is currently fourth-highest run-scorer in international cricket and stands second in the list of most international centuries scored. He also holds the record for scoring the most centuries in One Day International cricket.[4][5] Kohli was a member of the Indian team that won the 2011 Cricket World Cup and 2013 ICC Champions Trophy.

In 2013, Kohli was ranked number one in the ICC rankings f

In [35]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2753
Prompt Tokens:2078
Completion Tokens:675
Total Cost:0.0044670000000000005


In [36]:
response

{'text': "Virat Kohli (Hindi pronunciation; born 5 November 1988) is an Indian international cricketer and the former captain of the Indian national cricket team. He currently represents Royal Challengers Bangalore in the IPL and Delhi in domestic cricket. Kohli is widely regarded as one of the greatest batsmen in the history of cricket, and the best of this era.[3] He is the highest run scorer in T20I and IPL. In 2020, the International Cricket Council named him the male cricketer of the decade. Kohli is currently fourth-highest run-scorer in international cricket and stands second in the list of most international centuries scored. He also holds the record for scoring the most centuries in One Day International cricket.[4][5] Kohli was a member of the Indian team that won the 2011 Cricket World Cup and 2013 ICC Champions Trophy.\n\nIn 2013, Kohli was ranked number one in the ICC rankings for ODI batsmen. In 2018, he was ranked top Test batsman, making him the only Indian cricketer to

In [38]:
quiz=response.get('quiz')

In [41]:
quiz=json.loads(quiz)

In [42]:
quiz_table=[]
for key,value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [43]:
quiz_table

[{'MCQ': 'Virat Kohli currently represents which team in the IPL?',
  'Choices': 'a: Mumbai Indians | b: Royal Challengers Bangalore | c: Chennai Super Kings | d: Kolkata Knight Riders',
  'Correct': 'b'},
 {'MCQ': 'In which year was Virat Kohli named the male cricketer of the decade by the International Cricket Council?',
  'Choices': 'a: 2010 | b: 2015 | c: 2020 | d: 2025',
  'Correct': 'c'},
 {'MCQ': "What is Virat Kohli's ranking in terms of the highest run-scorer in international cricket?",
  'Choices': 'a: First | b: Second | c: Third | d: Fourth',
  'Correct': 'd'},
 {'MCQ': 'Which award did Virat Kohli win in 2017 and 2018, given to the ICC Cricketer of the Year?',
  'Choices': 'a: Sir Garfield Sobers Trophy | b: Arjuna Award | c: Padma Shri | d: Khel Ratna award',
  'Correct': 'a'},
 {'MCQ': 'In which year was Virat Kohli recognized as the ICC ODI Player of the Year?',
  'Choices': 'a: 2010 | b: 2012 | c: 2015 | d: 2018',
  'Correct': 'b'}]

In [44]:
quiz=pd.DataFrame(quiz_table)

In [45]:
quiz.to_csv('cricketmcq.csv',index=False)